# LAB 05 - Data preprocessing 3

## 05 스케일링


🧪 연습문제 — 🚀 직원 성과 분석을 위한 데이터 스케일링
- 한 회사의 인사팀에서는 직원들의 성과를 공정하게 평가하고 비교하기 위한 시스템을 구축하고 있습니다.

-  각 직원의 성과 지표(예: 월별 매출, 고객 만족도, 프로젝트 투입 시간 등)는 측정 단위와 범위가 서로 달라 직접 비교하기 어렵습니다. 따라서
  
-  이러한 지표들을 동일한 기준으로 변환하여, 모든 직원의 성과를 하나의 척도로 비교 및 분석할 수 있도록 데이터 스케일링 기법이 필요합니다.

- 여러분은 데이터 분석가로서 인사팀을 도와 다양한 성과 지표들을 적절한 스케일링 기법을 사용하여 표준화하는 작업을 수행해야 합니다.

📌 performance 데이터에 대해 다음 스케일링 기법들을 각 컬럼에 적용하세요.

1. MonthlySales 컬럼에는 Min-Max 스케일링을 적용하여 0과 1 사이의 값으로 변환하세요.

2. CustomerSatisfaction 컬럼에는 MaxAbs 스케일링을 적용하여 -1과 1 사이의 값으로 변환하세요.

3. ProjectHours 와 InnovationScore 컬럼에는 Standard 스케일링(표준화)을 적용하여 평균이 0, 표준편차가 1인 값으로 변환하세요.

4. 모든 스케일링 과정이 끝난 후, MonthlySales(Min-Max 스케일링), CustomerSatisfaction(MaxAbs 스케일링),
ProjectHours(Standard 스케일링), InnovationScore(Standard 스케일링) 컬럼의 값을 모두 더했을 때,
합산 값이 가장 높은 직원의 EmployeeID 를 찾아주세요.

In [ ]:
#라이브러리 참조
from hossam import load_data
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler,StandardScaler,RobustScaler
from pandas import merge
from pandas import DataFrame

In [ ]:
performance = load_data('performance')
performance

In [ ]:
# 1. MonthlySales 컬럼에는 Min-Max 스케일링을 적용하여 0과 1 사이의 값으로 변환하세요.
minmax_df = performance.copy()

MS = minmax_df[['MonthlySales']]

#MonthlySales 에 Sklearn 스케일링 적용
scaler = MinMaxScaler()
minmax_df['MonthlySales_MinMax'] = scaler.fit_transform(MS)
minmax_df

In [ ]:
# 2. CustomerSatisfaction 컬럼에는 MaxAbs 스케일링을 적용하여 -1과 1 사이의 값으로 변환하세요.

abs_df = performance.copy()

abs_scaler = MaxAbsScaler()
CS = abs_df[['CustomerSatisfaction']]
abs_df['CS_MaxAbs'] = abs_scaler.fit_transform(CS)
abs_df



In [ ]:
# 3. ProjectHours 와 InnovationScore 컬럼에는 Standard 스케일링(표준화)을 적용하여 평균이 0, 표준편차가 1인 값으로 변환하세요

std_df = performance.copy()

std_scaler = StandardScaler()
PH = std_df[['ProjectHours']]
IS = std_df[['InnovationScore']]


std_df['PH_STD'] = std_scaler.fit_transform(PH)
std_df['IS_STD'] = std_scaler.fit_transform(IS)

std_df


In [ ]:
#필요한 행들만 필터링

std_filtered= std_df.filter(['PH_STD','IS_STD'])
minmax_filtered=minmax_df.filter(['MonthlySales_MinMax'])
abs_filtered = abs_df.filter(['CS_MaxAbs'])

In [ ]:
std_minmax_merged= merge(std_filtered,minmax_filtered,left_index=True,right_index=True)

total_merged= merge(std_minmax_merged,abs_filtered,left_index=True,right_index=True)
total_merged

In [ ]:
a=DataFrame(total_merged.sum(axis=1))
b=a.rename(columns={0:'TotalScore'})
b

In [ ]:
c=b.reset_index()
c

In [ ]:
result = c.query('TotalScore==TotalScore.max()')
result

<강사님 풀이>
# 1. 데이터 복사 및 개별 컬럼 스케일링 (MinMax, MaxAbs)
df = performance_df.copy()
df['MonthlySales_scaled'] = MinMaxScaler().fit_transform(df[['MonthlySales']])
df['CustomerSatisfaction_scaled'] = MaxAbsScaler().fit_transform(df[['CustomerSatisfaction']])
df.head()

# 2. 복수 컬럼 Standard 스케일링 적용 후 분리 할당
scaled_standard_features = StandardScaler().fit_transform(df[['ProjectHours', 'InnovationScore']])
df['ProjectHours_scaled'] = scaled_standard_features[:, 0]
df['InnovationScore_scaled'] = scaled_standard_features[:, 1]
df.head()

# 3. 모든 스케일링된 점수 합산
df['TotalScaledScore'] = df['MonthlySales_scaled'] + df['CustomerSatisfaction_scaled'] + df['ProjectHours_scaled'] + df['InnovationScore_scaled']
df.head()

# 4. 합산 점수가 가장 높은 상위 1명 추출
max_data = df.sort_values('TotalScaledScore', ascending=False).head(1)
max_data

# 5. 가장 높은 점수를 받은 직원의 인덱스(ID) 확인
max_data.index[0]